<a href="https://colab.research.google.com/github/abhiyoke/HPC-Prac/blob/main/mpi_quick_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this in a Colab cell first to set up MPI
!apt-get install -y openmpi-bin libopenmpi-dev
!pip install mpi4py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-bin set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458270 sha256=143b90c2acb621be17287b359bb3297e3577f412bb8b15b5cbc6a603047d5640
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [ ]:
%%writefile mpi_quicksort.cpp
#include <bits/stdc++.h>
#include <mpi.h>
using namespace std;

void quick_sort(vector<int>& arr, int left, int right) {
    if (left >= right) return;
    int pivot = arr[right], i = left;
    for (int j = left; j < right; j++)
        if (arr[j] <= pivot) std::swap(arr[i++], arr[j]);
    std::swap(arr[i], arr[right]);
    quick_sort(arr, left, i-1);
    quick_sort(arr, i+1, right);
}

vector<int> merge(const std::vector<int>& a, const std::vector<int>& b) {
    vector<int> result;
    int i = 0, j = 0;
    while (i < a.size() && j < b.size()) {
        if (a[i] < b[j]) {
            result.push_back(a[i++]);
        } else {
            result.push_back(b[j++]);
        }
    }
    while (i < a.size()) result.push_back(a[i++]);
    while (j < b.size()) result.push_back(b[j++]);
    return result;
}

int main() {
    MPI_Init(NULL, NULL);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int N = 20;
    vector<int> data(N), local(N/size);

    // Root process generates data
    if (rank == 0) {
        for (int i = 0; i < N; i++) {
            data[i] = rand() % 100;
        }
        cout << "Unsorted: ";
        for (int i = 0; i < 10; i++) std::cout << data[i] << " ";
        cout << "...\n";
    }

    // Scatter data
    MPI_Scatter(data.data(), N/size, MPI_INT,
                local.data(), N/size, MPI_INT,
                0, MPI_COMM_WORLD);

    // Each process sorts its portion
    quick_sort(local, 0, local.size()-1);

    // Gather sorted portions
    MPI_Gather(local.data(), N/size, MPI_INT,
               data.data(), N/size, MPI_INT,
               0, MPI_COMM_WORLD);

    // Root merges results using custom merge function
    if (rank == 0) {
        vector<int> sorted;
        for (int i = 0; i < size; i++) {
            auto start = data.begin() + i*(N/size);
            auto end = start + (N/size);
            sorted = merge(sorted, std::vector<int>(start, end));
        }

        cout << "Sorted: ";
        for (int i = 0; i < 10; i++) std::cout << sorted[i] << " ";
        cout << "...\n";
    }

    MPI_Finalize();
    return 0;
}

Overwriting mpi_quicksort.cpp


In [ ]:
!mpic++ mpi_quicksort.cpp -o output_file_name
!mpirun --allow-run-as-root -np 1 ./output_file_name

Unsorted: 83 86 77 15 93 35 86 92 49 21 ...
Sorted: 15 21 26 26 27 35 36 40 49 59 ...
